<a href="https://colab.research.google.com/github/mohitkhannanu/ml-01/blob/master/Credit_Card_Fraud_Bayesian_xgboost_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bayesian-optimization

In [36]:
# Importing Libraries
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from datetime import datetime

import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Bayesian Optimization
#Importing necessary libraries
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [37]:
# Loading Dataset
data = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [38]:
# split data into X and y
X = data.iloc[:,1:29]
Y_cont = data.iloc[:,29]
Y_class = data.iloc[:,30]

In [39]:
# create 3 datasets - In Sample, Hold out, Validation data
# 70%, 15%, 15%
# In Sample

X_train, X_2, Y_cont_train, Y_cont_2, Y_class_train, Y_class_2 = train_test_split(X, Y_cont, Y_class, test_size=0.3, random_state=7)

#X_train.shape #(199364, 28)
#X_2.shape #(85443, 28)
#Y_cont_train.shape #(199364,)
#Y_cont_2.shape #(85443,)
#Y_class_train.shape #(199364, 28)
#Y_class_2.shape #(85443,)

X_hold_out, X_val, Y_cont_hold_out, Y_cont_val, Y_class_hold_out, Y_class_val = train_test_split(X_2, Y_cont_2, Y_class_2, test_size=0.5, random_state=7)

#X_hold_out.shape #(42721, 28)
#X_val.shape #(42722, 28)
#Y_cont_hold_out.shape #(42721,)
#Y_cont_val.shape #(42722,)
#Y_class_hold_out.shape #(42721, 28)
#Y_class_val.shape #(42722,)

In [58]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
starttime = datetime.now()
def bo_tune_xgb(max_depth, n_estimators, learning_rate):
    """
    params = {'max_depth': int(max_depth),
              #'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              #'subsample': 0.8,
              #'eta': 0.1,
              'eval_metric': 'error'}
    """
    test_model = XGBClassifier(booster='gbtree',
                  learning_rate=learning_rate, max_depth=int(max_depth),
                  n_estimators=int(n_estimators), n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  seed=None,
                  silent=None,verbosity=1)
    test_model.fit(X_train, Y_class_train)

    #Return the accuracy
    y_pred  = test_model.predict(X_hold_out)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    #accuracy = accuracy_score(Y_class_hold_out, predictions)
    ngini= gini_normalized(Y_class_hold_out, predictions)
    return ngini
    
#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (2,10),
                                             #'gamma': (0, 1),
                                             'learning_rate':(0.05, 0.3),
                                             'n_estimators':(100,600)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=15, init_points=10, acq='ei')

#Extracting the best parameters
params = xgb_bo.max['params']
print(params)
print(datetime.now() - starttime) #1:16:14.870691

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
|  1        |  0.8195   |  0.0899   |  2.434    |  235.9    |
|  2        |  0.8195   |  0.1029   |  2.638    |  284.0    |
|  3        |  0.8296   |  0.2508   |  7.963    |  280.1    |
|  4        |  0.8195   |  0.221    |  4.159    |  331.7    |
|  5        |  0.8481   |  0.2353   |  7.095    |  510.3    |
|  6        |  0.8481   |  0.2317   |  9.829    |  193.7    |
|  7        |  0.838    |  0.227    |  8.842    |  581.7    |
|  8        |  0.8195   |  0.1106   |  3.161    |  524.1    |
|  9        |  0.8195   |  0.1597   |  2.415    |  148.6    |
|  10       |  0.8195   |  0.2431   |  5.032    |  529.7    |
|  11       |  0.8195   |  0.2995   |  9.882    |  447.3    |
|  12       |  0.8481   |  0.1637   |  7.066    |  192.4    |
|  13       |  0.838    |  0.2432   |  9.935    |  597.9    |
|  14       |  0.838    |  0.2258   |  9.98     |  489.7    |
|  15   

In [59]:
#test accuracy
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.23168447688876614, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=194, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
model.fit(X_train, Y_class_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.23168447688876614, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=194, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [60]:
# make predictions for test data
y_pred  = model.predict(X_val)
#y_pred  = model.predict(X_hold_out)
#y_pred  = model.predict(X_train)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_class_val, predictions)
#accuracy = accuracy_score(Y_class_hold_out, predictions)
#accuracy = accuracy_score(Y_class_train, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.96%


In [61]:
# Gini definition
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

In [62]:
# Gini calculations
gini_predictions = gini(Y_class_val, predictions)
gini_max = gini(Y_class_val, Y_class_val)
ngini= gini_normalized(Y_class_val, predictions)

#gini_predictions = gini(Y_class_hold_out, predictions)
#gini_max = gini(Y_class_hold_out, Y_class_hold_out)
#ngini= gini_normalized(Y_class_hold_out, predictions)

#gini_predictions = gini(Y_class_train, predictions)
#gini_max = gini(Y_class_train, Y_class_train)
#ngini= gini_normalized(Y_class_train, predictions)

print('Gini: %.3f, Max. Gini: %.3f, Normalized Gini: %.3f' % (gini_predictions, gini_max, ngini))

# Grid Search results
# In Sample------------------ 0.816
# Hold Out------------------- 0.842
# Val-------------------------0.783

# Bayesian
# Val-------------------------0.793

Gini: 0.396, Max. Gini: 0.499, Normalized Gini: 0.793


In [ ]:
# Gini Drop Hold Out to Val
(0.783/0.842-1)*100